In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision
import torch.autograd as autograd
import skimage.feature
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import imp
import os
import sys
import math
import time
import random
import shutil

import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('bmh')
import PIL
from PIL import Image, ImageDraw, ImageFilter

%matplotlib inline

## Config

In [ ]:
DATA_PATH='/media/bfortuner/bigguy/data/sealions/sample/'
TRAIN_PATH=DATA_PATH+'train/'
TRAIN_DOTTED_PATH=DATA_PATH+'traindotted/'
TEST_PATH=DATA_PATH+'test/'
RESULTS_PATH=DATA_PATH+'results/'
WEIGHTS_PATH=DATA_PATH+'models/'
IMAGE_PATH=DATA_PATH+'images/'
PROJECT_NAME='sealions'

## Metadata

In [ ]:
train_data = pd.read_csv(DATA_PATH+'train.csv')
#sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
# List of images with the correct counts
print(train_data.head(6))

In [ ]:
# Notes
# cls -- sea lion class 
# tid -- train, train dotted, or test image id 
# _nb -- short for number
# x, y -- don't forget image arrays organized row, col, channels

cls_nb = 5
        
cls_names = (
    'adult_males',
    'subadult_males',
    'adult_females',
    'juveniles',
    'pups',
    'NOT_A_SEA_LION')

cls_ids = [0,1,2,3,4,5]

cls_names_to_id = {
    "adult_males":0,
    "subadult_males":1,
    "adult_females":2,
    "juveniles":3,
    "pups":4,
    "NOT_A_SEA_LION":5
}

# Average actual color of dot centers.
cls_colors = (
    (243,8,5),          # red
    (244,8,242),        # magenta
    (87,46,10),         # brown 
    (25,56,176),        # blue
    (38,174,21),        # green
    )

train_nb = 947        
test_nb = 18636
        
paths = {
    # Source paths
    'sample'     : os.path.join(DATA_PATH, 'sample_submission.csv'),
    'counts'     : os.path.join(DATA_PATH, '', 'train.csv'),
    'train'      : os.path.join(DATA_PATH, 'train', '{tid}.jpg'),
    'dotted'     : os.path.join(DATA_PATH, 'traindotted', '{tid}.jpg'),
    'test'       : os.path.join(DATA_PATH, 'test', '{tid}.jpg'),
    # Data paths
    'coords'     : os.path.join(DATA_PATH, 'coords.csv'),  
}

dot_radius = 3

bad_train_ids = (
    3, 7, 9, 21, 30, 34, 71, 81, 89, 97, 151, 184, 215, 234, 242, 
    268, 290, 311, 331, 344, 380, 384, 406, 421, 469, 475, 490, 499, 
    507, 530, 531, 605, 607, 614, 621, 638, 644, 687, 712, 721, 767, 
    779, 781, 794, 800, 811, 839, 840, 869, 882, 901, 903, 905, 909, 
    913, 927, 946)

## Load and Prepare Data

In [ ]:
def train_ids():
    """List of all valid train ids"""
    tids = range(0, train_nb)
    tids = list(set(tids) - set(bad_train_ids) )  # Remove bad ids
    tids.sort()
    return tids

def trainsmall_ids():
    return [i for i in range(41,51)]        # TrainSmall

def get_trn_fpaths(sample=True):
    fpaths,fnames = [],[]
    if sample:
        ids = trainsmall_ids()
    else:
        ids = train_ids()
    fpaths = []
    for id in ids:
        fpaths.append(DATA_PATH+'train/'+str(id)+'.jpg')
        fnames.append(str(id)+'.jpg')
    return fpaths,fnames

def test_ids():
    return [i for i in range(0, test_nb)]

def path(name, **kwargs):
    """Return path to various source files"""
    path = paths[name].format(**kwargs)
    return path

def counts() :
    """A map from train_id to list of sea lion class counts"""
    counts = {}
    fn = path('counts')
    with open(fn) as f:
        f.readline()
        for line in f:
            tid_counts = list(map(int, line.split(',')))
            counts[tid_counts[0]] = tid_counts[1:]
    return counts

In [ ]:
def _load_image(itype, tid, border=0):
    fn = path(itype, tid=tid)
    img = np.asarray(Image.open(fn))
    if border:
        height, width, channels = img.shape
        bimg = np.zeros( shape=(height+border*2, width+border*2, channels), dtype=np.uint8)
        bimg[border:-border, border:-border, :] = img
        img = bimg
    return img

def load_dotted_image(train_id, border=0):
    return _load_image('dotted', train_id, border)

def load_test_image(test_id, border=0):    
    return _load_image('test', test_id, border)

def load_train_image(train_id, border=0, mask=False):
    """Return image as numpy array

    border -- add a black border of this width around image
    mask -- If true mask out masked areas from corresponding dotted image
    """
    img = _load_image('train', train_id, border)
    if mask:
        # The masked areas are not uniformly black, presumable due to 
        # jpeg compression artifacts
        dot_img = _load_image('dotted', train_id, border).astype(np.uint16).sum(axis=-1)
        img = np.copy(img)
        img[dot_img<40] = 0
    return img

def get_paths_to_files(dir_path):
    filepaths = []
    fnames = []
    for (dirpath, dirnames, filenames) in os.walk(dir_path):
        filepaths.extend(os.path.join(dirpath, f) for f in filenames)
        fnames.extend([f for f in filenames])
    return filepaths, fnames

def get_random_image_path(dir_path):
    filepaths = get_paths_to_files(dir_path)[0]
    return filepaths[random.randrange(len(filepaths))]

def plot_img_array(img_arr, shape=(16,16)):
    # Plot numpy array image
    plt.figure(figsize=shape)
    plt.imshow(img_arr.astype('uint8'))
    plt.show()
    
def get_random_train_id(sample=True):
    if sample:
        trn_ids = trainsmall_ids()
    else:
        trn_ids = train_ids()
    rand_id = trn_ids[random.randrange(len(trn_ids))]
    return rand_id

def get_random_train_img_arr(sample=True):
    trn_id = get_random_train_id(sample)
    return load_train_image(trn_id)

In [ ]:
#Example Images
trn_img = load_train_image(41)
plot_img_array(trn_img)

In [ ]:
trn_id = get_random_train_id(True)
trn_arr = load_train_image(trn_id)
plot_img_array(trn_arr, shape=(8,8))

In [ ]:
trn_arr = get_random_train_img_arr(True)
plot_img_array(trn_arr, shape=(8,8))

In [ ]:
#Dotted Image
trn_id = get_random_train_id(True)
dot_arr = load_dotted_image(trn_id)
plot_img_array(dot_arr, shape=(8,8))

In [ ]:
#Dotted Image
trn_id = get_random_train_id(True)
trn_w_mask_arr = load_train_image(trn_id, mask=True)
plot_img_array(trn_w_mask_arr, shape=(8,8))

## Metrics

In [ ]:
def rmse(tid_counts, true_counts) :

    error = np.zeros(shape=[5] )

    for tid in tid_counts:
        true_counts = counts[tid]
        obs_counts = tid_counts[tid]
        diff = np.asarray(true_counts) - np.asarray(obs_counts)
        error += diff*diff
    print(error)
    error /= len(tid_counts)
    rmse = np.sqrt(error).sum() / 5
    return rmse

## Extract Coordinates

In [ ]:
trn_fpaths,trn_fnames = get_trn_fpaths(TRAIN_PATH)
coordinates_df = pd.DataFrame(index=trn_fnames, columns=cls_names)

In [ ]:
for filename in trn_fnames:
    print(TRAIN_DOTTED_PATH + filename)
    print(TRAIN_PATH + filename)
    # read the Train and Train Dotted images
    image_1 = cv2.imread(TRAIN_DOTTED_PATH + filename)
    image_2 = cv2.imread(TRAIN_PATH + filename)

    
    # absolute difference between Train and Train Dotted
    image_3 = cv2.absdiff(image_1,image_2)
    
    # mask out blackened regions from Train Dotted
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 20] = 0
    mask_1[mask_1 > 0] = 255
    
    mask_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    mask_2[mask_2 < 20] = 0
    mask_2[mask_2 > 0] = 255
    
    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_1)
    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_2) 
    
    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_3 = cv2.cvtColor(image_3, cv2.COLOR_BGR2GRAY)
    
    # detect blobs
    blobs = skimage.feature.blob_log(image_3, min_sigma=3, max_sigma=4, num_sigma=1, threshold=0.02)
    
    adult_males = []
    subadult_males = []
    pups = []
    juveniles = []
    adult_females = [] 
    not_sea_lions = [] 
    
    for blob in blobs:
        # get the coordinates for each blob
        y, x, s = blob
        # get the color of the pixel from Train Dotted in the center of the blob
        g,b,r = image_1[int(y)][int(x)][:]
        
        # decision tree to pick the class of the blob by looking at the color in Train Dotted
        if r > 200 and g < 50 and b < 50: # RED
            adult_males.append((int(x),int(y)))        
        elif r > 200 and g > 200 and b < 50: # MAGENTA
            subadult_males.append((int(x),int(y)))         
        elif r < 100 and g < 100 and 150 < b < 200: # GREEN
            pups.append((int(x),int(y)))
        elif r < 100 and  100 < g and b < 100: # BLUE
            juveniles.append((int(x),int(y))) 
        elif r < 150 and g < 50 and b < 100:  # BROWN
            adult_females.append((int(x),int(y)))
        else:  # BROWN
            not_sea_lions.append((int(x),int(y)))
            
    coordinates_df["adult_males"][filename] = adult_males
    coordinates_df["subadult_males"][filename] = subadult_males
    coordinates_df["adult_females"][filename] = adult_females
    coordinates_df["juveniles"][filename] = juveniles
    coordinates_df["pups"][filename] = pups
    coordinates_df["NOT_A_SEA_LION"][filename] = not_sea_lions

## Extract 32x32 images

In [ ]:
x = []
y = []

for filename in trn_fnames:    
    image = cv2.imread(TRAIN_PATH + filename)
    for lion_class in cls_names:
        for coordinates in coordinates_df[lion_class][filename]:
            thumb = image[coordinates[1]-16:coordinates[1]+16,coordinates[0]-16:coordinates[0]+16,:]
            if np.shape(thumb) == (32, 32, 3):
                x.append(thumb)
                y.append(lion_class)
x = np.array(x)
y = np.array(y)

## Plot Examples

In [ ]:
for lion_class in cls_names:
    f, ax = plt.subplots(1,10,figsize=(12,1.5))
    f.suptitle(lion_class)
    axes = ax.flatten()
    j = 0
    for a in axes:
        a.set_xticks([])
        a.set_yticks([])
        for i in range(j,len(x)):
            if y[i] == lion_class:
                j = i+1
                a.imshow(cv2.cvtColor(x[i], cv2.COLOR_BGR2RGB))
                break

## Encode Class Names

We convert the text class names into numerical ids

In [ ]:
y = np.array([cls_names_to_id[n] for n in y])
y[:200]

## Normalize and Load Data

In [ ]:
import scipy.misc

def load_img_as_np_arr(img_path):
    return scipy.misc.imread(img_path) #scipy

def get_mean_std_dataset(dir_path, sample_size=5):
    fpaths, fnames = get_paths_to_files(dir_path)
    total_mean = np.array([0.,0.,0.])
    total_std = np.array([0.,0.,0.]) 
    for f in fpaths[:sample_size]:
        img_arr = load_img_as_np_arr(f)
        mean = np.mean(img_arr, axis=(0,1))
        std = np.std(img_arr, axis=(0,1))
        total_mean += mean
        total_std += std
    avg_mean = total_mean / sample_size
    avg_std = total_std / sample_size
    print("mean: {}".format(avg_mean))
    print("stdev: {}".format(avg_std))
    return avg_mean, avg_std

def norm_meanstd(t, mean, std):
    """The Pytorch transforms module does this
    Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor
    channel = (channel - mean) / std"""
    return (t-mean) / std

def denorm_meanstd(t, mean, std):
    return (t * std) + mean

In [ ]:
IMG_MEAN, IMG_STD = get_mean_std_dataset(TRAIN_PATH,5)

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=IMG_MEAN, std=IMG_STD)
])

In [ ]:
print(x.shape)
norm_x = norm_meanstd(x,IMG_MEAN,IMG_STD)
norm_x = np.moveaxis(norm_x, 3, 1)
print (norm_x.shape)
print(x.mean((0,1,2)))
print(norm_x.mean((0,1,2)))

## Train/Test Split

In [ ]:
def get_trn_tst_split(x,y):
    train_size = 1000
    idx = [i for i in range(len(x))]
    random.shuffle(idx)
    trn_idx = idx[:train_size]
    tst_idx = idx[train_size:]
    train_x = np.array([x[i] for i in trn_idx])
    train_y = np.array([y[i] for i in trn_idx])
    test_x = np.array([x[i] for i in tst_idx])
    test_y = np.array([y[i] for i in tst_idx])
    return train_x, train_y, test_x, test_y

print(norm_x.shape)
train_x, train_y, test_x, test_y = get_trn_tst_split(norm_x,y)
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

In [ ]:
trn_dset = torch.utils.data.TensorDataset(torch.from_numpy(train_x).float(),torch.from_numpy(train_y).long())
test_dset = torch.utils.data.TensorDataset(torch.from_numpy(test_x).float(),torch.from_numpy(test_y).long())
print(trn_dset.data_tensor.size())
print(trn_dset.target_tensor.size())
print(test_dset.data_tensor.size())
print(test_dset.target_tensor.size())

In [ ]:
batch_size = 64
trn_dset_loader = torch.utils.data.DataLoader(trn_dset, batch_size=batch_size, shuffle=True)
test_dset_loader = torch.utils.data.DataLoader(trn_dset, batch_size=batch_size, shuffle=False)

## Build Model

### Model Init

* http://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                   kernel_size=5, padding=5//2) #padding='same'?
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1  = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, 
                   kernel_size=5, padding=5//2) #padding='same'?
        self.relu2 = nn.ReLU(inplace=True)

        self.fc1   = nn.Linear(in_features=64*16*16, out_features=120) 

        self.fc2   = nn.Linear(in_features=120, out_features=512)
        self.relu2 = nn.ReLU(inplace=True)
        self.drop2 = nn.Dropout(0.5)  

        self.fc3   = nn.Linear(in_features=512, out_features=6)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)

        # Flatten
        x = x.view(-1, self.num_flat_features(x))

        x = self.fc1(x)        
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### Loss/Optimizer

* http://pytorch.org/docs/optim.html?highlight=optimizer

In [ ]:
model = SimpleModel().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss().cuda()

## Train

In [ ]:
def train(model, train_loader, optimizer, criterion, trainF, epoch, projectName):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = Variable(inputs.cuda()), Variable(targets.cuda())
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        pred = get_predictions(output)
        train_err = error(pred, targets.data.cpu())
        partialEpoch = epoch + batch_idx / len(train_loader) - 1
        trainF.write('{},{},{}\n'.format(partialEpoch, loss.data[0], train_err))
        trainF.flush()
    print('Epoch {:d}: Train - Loss: {:.4f}\tErr: {:.4f}'.format(epoch, loss.data[0], train_err))
    return loss.data[0], train_err

def test(model, test_loader, criterion, testF=None, epoch=1):
    model.eval()
    test_loss = 0
    for data, target in test_loader:
        data, target = Variable(data.cuda(), volatile=True), Variable(target.cuda())
        output = model(data)
        test_loss += criterion(output, target).data[0]
    test_loss /= len(test_loader) #n_batches
    print('Test - Loss: {:.4f}'.format(test_loss))
    if testF:
        testF.write('{},{},{}\n'.format(int(epoch), test_loss))
        testF.flush()
    return test_loss

def predict(model, img_tensor):
    model.eval()
    data = Variable(img_tensor.cuda(), volatile=True)
    output = model(data)
    return output

In [ ]:
epochs = 1000

trn_loss_history = []
tst_loss_history = []
for epoch in range(epochs):  # loop over the dataset multiple times
    model.train()
    since = time.time()
    running_loss = 0.0
    for i, data in enumerate(trn_dset_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 500 == 499:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 500))
            trn_loss_history.append(running_loss / 500)
            running_loss = 0.0

            
    tst_loss = test(model,test_dset_loader,criterion,epoch=epoch)
    tst_loss_history.append(tst_loss)
    
    time_elapsed = time.time() - since 
    print('Time {:.0f}m {:.0f}s\n'.format(
        time_elapsed // 60, time_elapsed % 60))

    
            
print('Finished Training')

## Results

In [ ]:
plt.plot(tst_loss_history) #you can repeat this line with train loss to see both lines
plt.title('Testset loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')

In [ ]:
plt.plot(trn_loss_history) #you can repeat this line with train loss to see both lines
plt.title('Train loss')
plt.ylabel('loss')
plt.xlabel('epoch')

## Sliding Window

In [ ]:
predict_dset = datasets.ImageFolder(root=TRAIN_PATH, 
           transform=transforms.Compose([
            transforms.ToTensor(),
#             transforms.Normalize(torch.FloatTensor(IMG_MEAN), 
#                  torch.FloatTensor(IMG_STD)),
        ]))
predict_loader = torch.utils.data.DataLoader(
    predict_dset, batch_size=1, shuffle=False)

In [ ]:
image, target = next(iter(predict_loader))
image = image[0]
print(image.size())

def plot_tensor(tns_img, fs=(3,3)):
    "Takes a normalized tensor [0,1] and plots PIL image"
    pil_from_tns = transforms.ToPILImage()(tns_img)
    plt.figure(figsize=fs)
    plt.imshow(pil_from_tns)
    plt.show()

plot_tensor(image)

In [ ]:
def denorm255_np_arr(arr):
    return (arr * 255.).astype('uint8')

image = denorm255_np_arr(image.numpy())
print(image.shape)
image = np.moveaxis(image, 0, 2)
print(image.shape)
print(type(image))

In [ ]:
plot_img_array(image)

In [ ]:
x_test = []

for i in range(0,np.shape(image)[0],32):
    for j in range(0,np.shape(image)[1],32):                
        thumb = img[i:i+32,j:j+32,:]
        if np.shape(thumb) == (32,32,3):
            x_test.append(thumb)
    break

In [ ]:
plot_img_array(x_test[0],(3,3))

In [ ]:
slices_dset = torch.from_numpy(np.moveaxis(x_test, 3, 1)).float()
print(slices_dset.size())
print(slices_dset[0].size())

In [ ]:
preds = predict(model, slices_dset)
print(preds.size())
preds.cpu().max(0)
print (preds)

In [ ]:
values, indices = preds.cpu().max(1)
print(values.data.size(),indices.data.size())
print(indices[100:110])

In [ ]:
print(type(indices.data.numpy()))

In [ ]:
counts = np.unique(indices.data.numpy(), return_counts=True)
print(counts)

## Blob Detection

In [ ]:
# dataframe to store results in
fpaths,fnames = get_trn_fpaths()
# select a subset of files to run on
fnames = fnames

In [ ]:
count_df = pd.DataFrame(index=fnames, columns=cls_names).fillna(0)

In [ ]:
train_data[(train_data.train_id > 40) & (train_data.train_id < 51)]

In [ ]:
count_df

In [ ]:
image_1 = cv2.imread(os.path.join(TRAIN_PATH,fnames[3]))
image_2 = cv2.imread(os.path.join(TRAIN_DOTTED_PATH, fnames[3]))

In [ ]:
plot_img_array(image_1, shape=(20,20))

In [ ]:
image_2 = cv2.imread(os.path.join(TRAIN_PATH, fnames[0]))

mask_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
mask_2[mask_2 < 20] = 0
mask_2[mask_2 > 0] = 255

image_5 = cv2.bitwise_or(image_2, image_2, mask=mask_2) 

# convert to grayscale to be accepted by skimage.feature.blob_log
image_6 = cv2.cvtColor(image_5, cv2.COLOR_BGR2GRAY)

# detect blobs
blobs = skimage.feature.blob_log(image_6, min_sigma=3, max_sigma=4, num_sigma=1, threshold=0.02)

In [ ]:
for filename in fnames:
    image_2 = cv2.imread(os.path.join(TRAIN_PATH, filename))
    
    mask_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    mask_2[mask_2 < 20] = 0
    mask_2[mask_2 > 0] = 255
    
    image_5 = cv2.bitwise_or(image_2, image_2, mask=mask_2) 
    
    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_6 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    
    # detect blobs
    blobs = skimage.feature.blob_log(image_6, min_sigma=3, max_sigma=4, num_sigma=1, threshold=0.02)
    print
    # prepare the image to plot the results on
    image_7 = cv2.cvtColor(image_6, cv2.COLOR_GRAY2BGR)
        
    for blob in blobs:
        # get the coordinates for each blob
        y, x, s = blob
        cv2.circle(image_7, (int(x),int(y)), 8, (0,0,255), 2)            

    # output the results
          
    f, ax = plt.subplots(3,2,figsize=(10,16))
    (ax1, ax2, ax3, ax4, ax5, ax6) = ax.flatten()
    plt.title('%s'%filename)
    
    ax1.imshow(cv2.cvtColor(image_2[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax1.set_title('Train')
    ax4.imshow(cv2.cvtColor(image_5[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax4.set_title('Mask blackened areas of Train Dotted')
    ax5.imshow(image_6[700:1200,2130:2639], cmap='gray')
    ax5.set_title('Grayscale for input to blob_log')
    ax6.imshow(cv2.cvtColor(image_7[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax6.set_title('Result')

    plt.show()

In [ ]:
for filename in fnames:
    # read the Train and Train Dotted images
    image_1 = cv2.imread(os.path.join(TRAIN_DOTTED_PATH,filename))
    image_2 = cv2.imread(os.path.join(TRAIN_PATH, filename))
    
    # absolute difference between Train and Train Dotted
    image_3 = cv2.absdiff(image_1,image_2)
    
    # mask out blackened regions from Train Dotted
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 20] = 0
    mask_1[mask_1 > 0] = 255
    
    mask_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    mask_2[mask_2 < 20] = 0
    mask_2[mask_2 > 0] = 255
    
    image_4 = cv2.bitwise_or(image_3, image_3, mask=mask_1)
    image_5 = cv2.bitwise_or(image_4, image_4, mask=mask_2) 
    
    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_6 = cv2.cvtColor(image_5, cv2.COLOR_BGR2GRAY)
    
    # detect blobs
    blobs = skimage.feature.blob_log(image_6, min_sigma=3, max_sigma=4, num_sigma=1, threshold=0.02)
    
    # prepare the image to plot the results on
    image_7 = cv2.cvtColor(image_6, cv2.COLOR_GRAY2BGR)
    
    for blob in blobs:
        # get the coordinates for each blob
        y, x, s = blob
        # get the color of the pixel from Train Dotted in the center of the blob
        b,g,r = image_1[int(y)][int(x)][:]
        
        # decision tree to pick the class of the blob by looking at the color in Train Dotted
        if r > 200 and b < 50 and g < 50: # RED
            count_df["adult_males"][filename] += 1
            cv2.circle(image_7, (int(x),int(y)), 8, (0,0,255), 2)            
        elif r > 200 and b > 200 and g < 50: # MAGENTA
            count_df["subadult_males"][filename] += 1
            cv2.circle(image_7, (int(x),int(y)), 8, (250,10,250), 2)            
        elif r < 100 and b < 100 and 150 < g < 200: # GREEN
            count_df["pups"][filename] += 1
            cv2.circle(image_7, (int(x),int(y)), 8, (20,180,35), 2) 
        elif r < 100 and  100 < b and g < 100: # BLUE
            count_df["juveniles"][filename] += 1 
            cv2.circle(image_7, (int(x),int(y)), 8, (180,60,30), 2)
        elif r < 150 and b < 50 and g < 100:  # BROWN
            count_df["adult_females"][filename] += 1
            cv2.circle(image_7, (int(x),int(y)), 8, (0,42,84), 2)            
        else:
            count_df["NOT_A_SEA_LION"][filename] += 1
            cv2.circle(image_7, (int(x),int(y)), 8, (255,255,155), 2)
    
    # output the results
          
    f, ax = plt.subplots(3,2,figsize=(10,16))
    (ax1, ax2, ax3, ax4, ax5, ax6) = ax.flatten()
    plt.title('%s'%filename)
    
    ax1.imshow(cv2.cvtColor(image_2[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax1.set_title('Train')
    ax2.imshow(cv2.cvtColor(image_1[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax2.set_title('Train Dotted')
    ax3.imshow(cv2.cvtColor(image_3[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax3.set_title('Train Dotted - Train')
    ax4.imshow(cv2.cvtColor(image_5[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax4.set_title('Mask blackened areas of Train Dotted')
    ax5.imshow(image_6[700:1200,2130:2639], cmap='gray')
    ax5.set_title('Grayscale for input to blob_log')
    ax6.imshow(cv2.cvtColor(image_7[700:1200,2130:2639,:], cv2.COLOR_BGR2RGB))
    ax6.set_title('Result')

    plt.show()

## References

* https://www.kaggle.com/philschmidt/noaa-fisheries-steller-sea-lion-population-count/sea-lion-correlations-cv2-template-matching
* https://www.kaggle.com/radustoicescu/noaa-fisheries-steller-sea-lion-population-count/get-coordinates-using-blob-detection
* https://www.kaggle.com/asymptote/noaa-fisheries-steller-sea-lion-population-count/count-extract-sea-lions
* https://www.kaggle.com/radustoicescu/noaa-fisheries-steller-sea-lion-population-count/use-keras-to-classify-sea-lions-0-91-accuracy

**Satallite Unet Examples**
* https://www.kaggle.com/drn01z3/dstl-satellite-imagery-feature-detection/end-to-end-baseline-with-u-net-keras/code (similar view from above on small objects)
* https://www.kaggle.com/ceperaang/dstl-satellite-imagery-feature-detection/lb-0-42-ultimate-full-solution-run-on-your-hw/code
* https://www.kaggle.com/amanbh/dstl-satellite-imagery-feature-detection/visualize-polygons-and-image-data/code

**Spacenet challenge**
* https://github.com/SpaceNetChallenge/BuildingDetectors

**Blog Detection**
* http://scikit-image.org/docs/dev/api/skimage.feature.html